In [1]:
# Условие: создайте csv файл с таким содержимым:

# title,author,genre,sales,year
# "1984", "George Orwell", "Science Fiction", 5000, 1949
# "The Lord of the Rings", "J.R.R. Tolkien", "Fantasy", 3000, 1954
# "To Kill a Mockingbird", "Harper Lee", "Southern Gothic", 4000, 1960
# "The Catcher in the Rye", "J.D. Salinger", "Novel", 2000, 1951
# "The Great Gatsby", "F. Scott Fitzgerald", "Novel", 4500, 1925

# Задание:

# — Используя Spark прочитайте данные из файла csv.
# — Фильтруйте данные, чтобы оставить только книги, продажи которых превышают 3000 экземпляров.
# — Сгруппируйте данные по жанру и вычислите общий объем продаж для каждого жанра.
# — Отсортируйте данные по общему объему продаж в порядке убывания.
# — Выведите результаты на экран.

In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Practise').getOrCreate()

In [3]:
data=[("1984", "George Orwell", "Science Fiction", 5000, 1949),
("The Lord of the Rings", "J.R.R. Tolkien", "Fantasy", 3000, 1954),
("To Kill a Mockingbird", "Harper Lee", "Southern Gothic", 4000, 1960),
("The Catcher in the Rye", "J.D. Salinger", "Novel", 2000, 1951),
("The Great Gatsby", "F. Scott Fitzgerald", "Novel", 4500, 1925)
]
columns = ["title", "author", "genre", "sales", "year"]

In [4]:
# создание датафрейма
df = spark.createDataFrame(data, columns)

In [5]:
df.show()

+--------------------+-------------------+---------------+-----+----+
|               title|             author|          genre|sales|year|
+--------------------+-------------------+---------------+-----+----+
|                1984|      George Orwell|Science Fiction| 5000|1949|
|The Lord of the R...|     J.R.R. Tolkien|        Fantasy| 3000|1954|
|To Kill a Mocking...|         Harper Lee|Southern Gothic| 4000|1960|
|The Catcher in th...|      J.D. Salinger|          Novel| 2000|1951|
|    The Great Gatsby|F. Scott Fitzgerald|          Novel| 4500|1925|
+--------------------+-------------------+---------------+-----+----+



In [6]:
# создание файла с целевым содержимым
df.write.csv("./file.csv", header=True)

In [7]:
# чтение данных из файла csv.
df_pyspark = spark.read.load('file.csv', format="csv", sep=",", header="true", Infer_schema=True)

In [8]:
df_pyspark.show()

+--------------------+-------------------+---------------+-----+----+
|               title|             author|          genre|sales|year|
+--------------------+-------------------+---------------+-----+----+
|To Kill a Mocking...|         Harper Lee|Southern Gothic| 4000|1960|
|The Catcher in th...|      J.D. Salinger|          Novel| 2000|1951|
|    The Great Gatsby|F. Scott Fitzgerald|          Novel| 4500|1925|
|                1984|      George Orwell|Science Fiction| 5000|1949|
|The Lord of the R...|     J.R.R. Tolkien|        Fantasy| 3000|1954|
+--------------------+-------------------+---------------+-----+----+



In [9]:
# — Фильтруйте данные, чтобы оставить только книги, продажи которых превышают 3000 экземпляров
from pyspark.sql.functions import col, sum as sum_sp
df_filter = df_pyspark.filter(col("sales") > 3000)
df_filter.show()

+--------------------+-------------------+---------------+-----+----+
|               title|             author|          genre|sales|year|
+--------------------+-------------------+---------------+-----+----+
|To Kill a Mocking...|         Harper Lee|Southern Gothic| 4000|1960|
|    The Great Gatsby|F. Scott Fitzgerald|          Novel| 4500|1925|
|                1984|      George Orwell|Science Fiction| 5000|1949|
+--------------------+-------------------+---------------+-----+----+



In [10]:
# — меняем тип данных в sales на int
df_pyspark = df_pyspark.withColumn("sales", df_pyspark["sales"].cast('int'))
df_pyspark

DataFrame[title: string, author: string, genre: string, sales: int, year: string]

In [14]:
# — Сгруппируйте данные по жанру и вычислите общий объем продаж для каждого жанра
df_group = df_pyspark.groupBy("genre").agg(sum_sp("sales"))
df_group.show()

+---------------+----------+
|          genre|sum(sales)|
+---------------+----------+
|Southern Gothic|      4000|
|          Novel|      6500|
|        Fantasy|      3000|
|Science Fiction|      5000|
+---------------+----------+



In [15]:
# — Отсортируйте данные по общему объему продаж в порядке убывания.
# — Выведите результаты на экран.
df_group.orderBy(col('sum(sales)').desc()).show()

+---------------+----------+
|          genre|sum(sales)|
+---------------+----------+
|          Novel|      6500|
|Science Fiction|      5000|
|Southern Gothic|      4000|
|        Fantasy|      3000|
+---------------+----------+

